Data Visualization Lab 3

In [ ]:
pip install geopandas

Question 1 (Bubble map to show the populations of all the state capital of India)

In [3]:
# import required libraries
from urllib import request
import pandas as pd
import numpy as numpy
import folium
from shapely.geometry import Point
import geopandas as gpd
import webbrowser
import matplotlib.pyplot as plt
import requests

In [ ]:
# Import dataset
df=pd.read_csv("/content/lab3 - Sheet1.csv")
df2=df.copy()
# Using folium library create bubble map of India
contry_bubble_map=folium.Map(location=[21.53, 78.69], zoom_start=5.5)
for i, j in df.iterrows():
    folium.CircleMarker([j['latitude'], j['longitude']],radius=j['population']/500000,fill_color='skyblue',color='blue',fill=True,).add_child(folium.Popup(j['state'] + ": " + str(j['population']) + " people")).add_to(contry_bubble_map)

# Show bubble map
contry_bubble_map


In [ ]:
a=df2.longitude
b=df2.latitude
data_=gpd.GeoDataFrame(df2,geometry=[Point(points) for points in zip(a,b)])
map_bubble_india=folium.Map(location=[21.53, 78.69],zoom_start=5.5)

# Bubble map of India using shapely and folium
data_.apply(lambda x: folium.CircleMarker(location=[x.geometry.y, x.geometry.x],radius=x.population/500000,
                                        color='skyblue',fill_color='blue',fill=True,
                                       ).add_child(folium.Tooltip(f"Latitude: {x.latitude},Longitude:{x.longitude},Population:{x.population}"))
                                        .add_to(map_bubble_india),axis=1)
# Show bubble map
map_bubble_india

Question 2 ( Connection graph of Go Air domestic flights of India)

In [6]:
# Load Given dataset
df=pd.read_csv("/content/flight_data.csv")
print(df.head())
df.describe()

      Origin Destination Flight Number Departure Time Arrival Time  \
0  Abu Dhabi   Ahmedabad          G8 6          23:30        14:10   
1  Abu Dhabi   Ahmedabad          G8 6          23:30        14:35   
2  Abu Dhabi   Ahmedabad          G8 6          23:30        18:25   
3  Abu Dhabi   Ahmedabad          G8 6          23:30        08:55   
4  Abu Dhabi   Ahmedabad         G8 76          21:40        07:35   

  Flight Time  Routing      Operations       No Effective From EffectiveTo  
0       13:10  Via BOM              Sa   G8 247     02/03/2023  02/05/2023  
1       13:35  Via BOM  M,W,Th,F,Sa,Su  G8 2504     02/03/2023  02/05/2023  
2       17:25  Via BOM           Daily   G8 364     02/03/2023  02/05/2023  
3       07:55  Via BOM           Daily   G8 365     02/03/2023  02/05/2023  
4       08:25  Via DEL  M,Tu,W,Th,F,Su   G8 719     02/03/2023  02/11/2023  


,Origin,Destination,Flight Number,Departure Time,Arrival Time,Flight Time,Routing,Operations,No,Effective From,EffectiveTo
count,3244,3244,3244,3244,3244,3244,3244,3244,3244,3244,3244
unique,36,36,266,175,185,240,25,72,201,3,24
top,Mumbai,Mumbai,G8 2620,09:25,08:25,12:45,Via BOM,Daily,-,02/03/2023,02/15/2023
freq,348,328,55,67,103,38,1301,1176,367,2857,1308


In [7]:
# Unique Values
origin1=pd.unique(df['Origin'])
dest1 = pd.unique(df['Destination'])
cities=numpy.unique(numpy.concatenate((origin1,dest1),0))
cities

array(['Abu Dhabi', 'Ahmedabad', 'Amritsar', 'Bagdogra', 'Bangkok',
       'Bengaluru', 'Chandigarh', 'Chennai', 'Dammam', 'Dehra Dun',
       'Delhi', 'Dubai', 'Goa', 'Goa-New Goa', 'Guwahati', 'Hyderabad',
       'Jaipur', 'Jammu', 'Kannur', 'Kochi', 'Kolkata', 'Kuwait', 'Leh',
       'Lucknow', 'Male', 'Mumbai', 'Muscat', 'Nagpur', 'Patna', 'Phuket',
       'Port Blair', 'Pune', 'Ranchi', 'Srinagar', 'Thoise', 'Varanasi'],
      dtype=object)

In [8]:
latitudes=[]
longitudes=[]
API_KEY='c3e69fdf5a8d0bd03f0a25a05e48553b'
link='https://nominatim.openstreetmap.org/search?q={i}&format=json'

for i in cities:
   d_j=requests.get( link).json()
   lon=(d_j[0]['lon'])
   lat=(d_j[0]['lat'])
   latitudes.append(lat)
   longitudes.append(lon)



In [9]:
coord_lat={k:val for k,val in zip(cities, latitudes)}
coord_lon={k:val for k,val in zip(cities, longitudes)}

In [10]:
df['origin_lat']=df['Origin'].map(coord_lat)
df['origin_lon']=df['Origin'].map(coord_lon)
df['destination_lat']=df['Destination'].map(coord_lat)
df['destination_lon']=df['Destination'].map(coord_lon)


In [11]:
dict_value=df['Origin'].value_counts(dropna=False).to_dict()
df['origin_count']=df['Origin'].map(dict_value)

In [12]:
filename="data.csv"
df.to_csv(filename, index=False)

In [ ]:
df2=pd.read_csv("data.csv")
df2.head()

In [ ]:
# Connection Map using Folium
connection_map=folium.Map(location=[21.53, 78.69], zoom_start=5.5)
for i, j in df2.iterrows():
    circle_marker=folium.CircleMarker(location=[j['origin_lat'], j['origin_lon']],
                        radius=(j['origin_count']/10),color='blue',
                        fill=True,fill_color=None,)
    connection=circle_marker.add_child(folium.Tooltip(f"Place: {j['Origin']}, Longitude: {j['origin_lon']}, Latitude: {j['origin_lat']} \n Origin Count: {'origin_count'}"))
    connection.add_to(connection_map)
    map_folium=folium.PolyLine([[j['origin_lat'], j['origin_lon']], [j['destination_lat'], j['destination_lon']]], 
                  color='blue',opacity=0.2
                )
    final_map=map_folium.add_child(folium.Tooltip(f"Route: {j['Origin']} to {j['Destination']} \n Routing: {j['Routing']} \n Flight No: {j['Flight Number']}"))
    final_map.add_to(connection_map)
# Show connection map
connection_map